In [1]:
import torch
import pygame

pygame 2.5.1 (SDL 2.28.2, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# world initialization settings:
simulation_dimension = 2
num_of_particles = 50
scatter_size = 200

# particle interaction settings:
big_G = 1
particle_mass = 1
momentum_conservation_value = 1

In [3]:
GPU = torch.device("cuda")
world = (2 * scatter_size * torch.rand(num_of_particles * simulation_dimension, dtype=torch.float32, device=GPU) - scatter_size).view(-1, simulation_dimension)

In [4]:
def get_single_particle_vector(particle_idx, world):
    vector_sum = torch.tensor([0] * simulation_dimension, dtype=torch.float32, device=GPU)
    current_particle = world[particle_idx]
    COUNTER = 0
    
    for x in world:
        if COUNTER != particle_idx:
            difference = x - current_particle
            difference_squared = difference ** 2
            distance = torch.sum(difference_squared)
            inverse_distance = 1 / distance
            vector = difference * inverse_distance * big_G
            vector_sum += vector
        
        COUNTER += 1
    
    return vector_sum

In [5]:
def get_global_vectors(world, momentum_vectors):
    global_vectors = torch.zeros(world.shape, dtype=torch.float32, device=GPU)
    
    for x in range(len(world)):
        global_vectors[x] += get_single_particle_vector(x, world)
    
    return global_vectors + momentum_vectors * momentum_conservation_value

In [6]:
# Initialize Pygame
pygame.init()

# Set the window dimensions
window_size = (800, 800)

# Create a window
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Particle Simulation")

# Define colors
bg_color = (0, 0, 0)

# Function to map PyTorch tensor positions to screen coordinates
def map_to_screen(position, max_position):
    x, y = position.detach().cpu().numpy()
    x = int((x / max_position + 1) * window_size[0] / 2)
    y = int((y / max_position + 1) * window_size[1] / 2)
    return x, y

# Main game loop
running = True

vectors = torch.zeros(world.shape, dtype=torch.float32, device=GPU)#world * 3 + (torch.rand(world.shape, dtype=torch.float32, device=GPU) * 700 - 350)
max_position = 0

while running:
    if float(torch.max(torch.abs(world))) >= max_position:
        max_position = float(torch.max(torch.abs(world)))
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    screen.fill(bg_color)

    # Update your PyTorch world (you'll need to adapt this part)
    vectors = get_global_vectors(world, vectors)
    world += vectors
    
    if world.shape[1] < 2:
        zeros = torch.zeros((world.shape[0], 2 - world.shape[1]), device=GPU)
        clipped = torch.cat((world, zeros), dim=1)
    else:
        clipped = world[:, :2]

    counter = 0
    # Draw dots on the screen based on your PyTorch world
    for position in world:
        # dims 0->2
        if simulation_dimension >= 2:
            position = map_to_screen(world[counter][:2], max_position)
        else:
            zeros = torch.zeros((world.shape[0], 2 - world.shape[1]), device=GPU)
            position = map_to_screen(torch.cat((world, zeros), dim=1), max_position)
        
        # dims 2->5
        if simulation_dimension >= 5:
            color = tuple(((world[counter][2:5] - world[counter][2:5].min()) / (world[counter][2:5].max() - world[counter][2:5].min()) * 255).long().tolist())
        else:
            color = (255, 255, 255)
        
        pygame.draw.circle(screen, color, position, 1)#float(torch.sigmoid(torch.mean(world[counter][2:])) * particle_display_size + 1))  # 5 is the radius of the dot
        counter += 1
    #pygame.draw.circle(screen, (116, 255, 56), map_to_screen(torch.tensor([0, 0]), max_position), 5)  # 5 is the radius of the dot

    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()
sys.exit()

NameError: name 'sys' is not defined